In [30]:
from collections import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from sklearn import linear_model

# DATA CLEANING --same for all parts

file_path = "Top_1000_IMDb_movies_New_version.csv"
data = pd.read_csv(file_path, header=0)
clean_data = data.copy()
clean_data = clean_data.replace("#222", "72.5")

# convert format "II 2019" to "2019"
def convert_string(x):
    if x[0] == 'I':
        return x[-4:]
    else:
        return x
clean_data["Year of Release"] = clean_data["Year of Release"].apply(convert_string)

# convert votes to numbers
def convert_to_int(x):
    return int(x.replace(',', ''))
clean_data["Votes"] = clean_data["Votes"].apply(convert_to_int)

# calculate means
Wt_mean = clean_data["Watch Time"].mean()
Rating_mean = clean_data["Movie Rating"].mean()
score_mean = clean_data["Metascore of movie"].mean()
revenue_mean = clean_data["Gross"].apply(float).mean()
votes_mean = clean_data["Votes"].mean()

# Data filling with mean values
clean_data["Watch Time"] = clean_data["Watch Time"].fillna(Wt_mean)
clean_data["Movie Rating"] = clean_data["Movie Rating"].fillna(Rating_mean)
clean_data["Metascore of movie"] = clean_data["Metascore of movie"].fillna(score_mean)
clean_data["Gross"] = clean_data["Gross"].fillna(revenue_mean)
clean_data["Votes"] = clean_data["Votes"].fillna(votes_mean)


print("Shape before clean: ", data.shape)
print("Shape after clean: ", clean_data.shape)

# words and counts in each description
count = Counter()
desc_words = clean_data["Description"].str.split()
for i in desc_words:
    count.update(i)
desc_wc = clean_data["Description"].str.split().apply(len)

Shape before clean:  (1000, 9)
Shape after clean:  (1000, 9)


In [31]:
# I: Univariate Stats

YoR_summ = clean_data["Year of Release"].describe()
print("Year of Release Statistics: \n", YoR_summ, "\n")

Wt_summ = clean_data["Watch Time"].describe()
print("Watch Time Statistics: \n", Wt_summ, "\n")

Rating_summ = clean_data["Movie Rating"].describe()
print("Rating Statistics: \n", Rating_summ, "\n")

score_summ = clean_data["Metascore of movie"].describe()
print("Metascore Statistics: \n",score_summ, "\n")

revenue_summ = clean_data["Gross"].apply(float).describe()
print("Gross Revenue Statistics: \n", revenue_summ, "\n")

votes_summ = clean_data["Votes"].describe()
print("Voting Statistics: \n", votes_summ, "\n")

# words and counts in each description
avg_wc = desc_wc.mean()
std_wc = desc_wc.std()
quant_wc = desc_wc.quantile([.25, .5, .75])
print("Description Statistics:")
print("Average word count: ", round(avg_wc))
print("Std: ", round(std_wc))
print("Quantiles:\n", quant_wc)
print("\n")
print("Count dictionary: ", count)
print("50 Most popular words: ", count.most_common(50))
print("50 Least popular words: ", count.most_common()[-50:])

meta = clean_data["Metascore of movie"].apply(float).to_numpy()
rev = clean_data["Gross"].apply(float).to_numpy()
rating = clean_data["Movie Rating"].apply(float).to_numpy()
votes = clean_data["Votes"].apply(float).to_numpy()
print("\n")
print("Correlation of Metascores to Revenue: ")
print(np.corrcoef(meta, rev)[0])
print("Correlation of Ratings to Revenue: ")
print(np.corrcoef(rating, rev)[0])
print("\n")
print("Correlation of Metascores to Votes: ")
print(np.corrcoef(meta, votes)[0])
print("Correlation of Ratings to Votes: ")
print(np.corrcoef(rating, votes)[0])




Year of Release Statistics: 
 count     1000
unique     102
top       2014
freq        31
Name: Year of Release, dtype: object 

Watch Time Statistics: 
 count    1000.000000
mean      124.253000
std        28.800355
min        45.000000
25%       103.000000
50%       120.000000
75%       139.000000
max       321.000000
Name: Watch Time, dtype: float64 

Rating Statistics: 
 count    1000.000000
mean        7.970200
std         0.275732
min         7.600000
25%         7.800000
50%         7.900000
75%         8.100000
max         9.300000
Name: Movie Rating, dtype: float64 

Metascore Statistics: 
 count    1000.000000
mean       79.011834
std        11.005772
min        28.000000
25%        73.000000
50%        79.011834
75%        86.000000
max       100.000000
Name: Metascore of movie, dtype: float64 

Gross Revenue Statistics: 
 count    1000.000000
mean       72.837613
std       105.220384
min         0.000000
25%         5.010000
50%        44.805000
75%        72.837613
max    